In [1]:
#default_exp sklearn

In [ ]:
#TODO: add "toggle_cond" to HPToggle

In [2]:
#export
from hypster.oo_hp import *
from hypster.hypster_prepare import *

import fastai2
from fastai2.tabular.all import *
from fastai2.metrics import *

from sklearn.model_selection import train_test_split

from copy import deepcopy

import optuna

In [3]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer

In [4]:
from sklearn.impute import SimpleImputer

In [5]:
from sklearn.preprocessing import OneHotEncoder

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
from category_encoders import BinaryEncoder

In [8]:
#export
SEED = 42

# Read Data

In [9]:
#export
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('C:/Users/user/.fastai/data/adult_sample/adult.csv'),Path('C:/Users/user/.fastai/data/adult_sample/export.pkl'),Path('C:/Users/user/.fastai/data/adult_sample/models')]

In [10]:
#export
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [11]:
#export
df = df.sample(frac=0.1)

In [12]:
#export
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
dep_var = "salary"

In [13]:
y = np.where(df[dep_var] == ">=50k", 1, 0)
df.drop(dep_var, axis=1, inplace=True)

In [14]:
#export
train_df, test_df, y_train, y_test = train_test_split(df, y, test_size=0.6, 
                                                     random_state=SEED, 
                                                     stratify=y)

# Preprocessing

In [15]:
train_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
8643,82,?,29441,7th-8th,4.0,Widowed,?,Not-in-family,White,Male,0,0,5,United-States
9310,27,Private,221436,Bachelors,13.0,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,40,Cuba
24615,39,Local-gov,134367,HS-grad,9.0,Never-married,Adm-clerical,Own-child,White,Female,0,0,35,United-States
20029,57,Private,548256,12th,8.0,Married-civ-spouse,Adm-clerical,Husband,Black,Male,0,0,40,United-States
12595,23,Private,212888,Some-college,10.0,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States


In [16]:
RandomForestClassifier = prepare(RandomForestClassifier)
SimpleImputer = prepare(SimpleImputer)
Pipeline = prepare(Pipeline)
ColumnTransformer = prepare(ColumnTransformer)
FeatureUnion = prepare(FeatureUnion)

In [17]:
imputer = SimpleImputer(strategy=HpCategorical("imp_strategy", ["mean", "median"]))

In [18]:
cat_enc = BinaryEncoder()

In [19]:
rf = RandomForestClassifier(n_estimators=HpInt("n_trees", 10, 40, 10),
                            criterion=HpCategorical("criterion", ['gini', 'entropy']),
                            max_depth=HpToggle("max_depth_toggle", HpInt("max_depth", 5, 30, 5)),
                            max_features=HpCategorical("max_features", ['auto', 'sqrt', 'log2', None]),
                            max_leaf_nodes=HpToggle("max_leaf_nodes_tog", HpInt("max_leaf_nodes", 2, 20)),
                            min_impurity_decrease=0.0,
                            bootstrap=HpBool("bootstrap"),
                            oob_score=False,
                            n_jobs=-1,
                            random_state=SEED,
                            verbose=0,
                            warm_start=False,
                            class_weight=None,
                            ccp_alpha=0.0,
                            max_samples=None
                            )

In [21]:
rf = RandomForestClassifier(oob_score=HpBool("oob_score"),
                            criterion=HpToggle("tog_criterion",
                                               HpCategorical("criterion", ['gini', 'entropy'])), 
                                               cond="oob_score"==True
                                               )
                            
                            max_depth=HpToggle("max_depth_toggle", HpInt("max_depth", 5, 30, 5)),
                            
                            n_estimators=HpIfElse(condif="oob_score" > 5,
                                                  then=HpInt("n_trees", 10, 40, 10),
                                                  elsa=HpFloat("n_tress", 0.1, 0.8)
                                                 )
                            a : b ? c
                            cond1 = HpIfElse(
                                condif="oob_score" > 5,
                                then=[
                                    HpInt("n_trees", 10, 40, 10),
                                    HpInt()
                                ],
                                elsa=[
                                    HpInt("n_trees")
                                ]
                            )

                            criterion=HpCategorical("criterion", ['gini', 'entropy'], 
                                                    toggle_cond=Hp("oob_score")==True
                                                    
                                                    toggle_cond=HpGeq("oob_score", 4) & HpGeq("oob_score2", 7)
                                                    toggle_cond=HpEq("oob_score", True)
                                                    toggle_cond=HpLeq("oob_score", 4)
                                                   )
                            
                            max_depth=HpInt("max_depth", 5, 30, 5, toggle_cond="oob_score"==True)
                            )

SyntaxError: invalid syntax (<ipython-input-21-629a769970db>, line 9)

In [ ]:
cont_imp  = ColumnTransformer([("imp", imputer, cont_names)])

In [ ]:
cat_enc   = ColumnTransformer([("cat_enc", cat_enc, cat_names)])

In [ ]:
fe = FeatureUnion([("cat", cat_enc), ("cont", cont_imp)])

In [ ]:
pipe = Pipeline([("fe", fe), ("model", rf)])

In [ ]:
#export
import datetime
def run_learner(fit_method, get_metric, n_trials=5): #learner
    class Objective():
        def __init__(self, fit_method, get_metric): #learner
            #self.learner   = learner
            self.fit_method = fit_method
            self.get_metric = get_metric
            
        def __call__(self, trial): 
            #learner = self.learner.sample(trial)
            self.fit_method.sample(trial)
            res = self.get_metric.sample(trial)
            #print(self.fit_method.base_call)
            #print(self.get_metric.base_call.base_call)
            print(res)
            return res

    objective = Objective(fit_method, get_metric) #learner
    optuna.logging.set_verbosity(0)
    pruner = optuna.pruners.NopPruner()
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    study = optuna.create_study(direction="maximize", study_name = now, pruner=pruner)
    study.optimize(objective, n_trials=n_trials, n_jobs=1, timeout=600)
    return study

In [ ]:
#export
study = run_learner(#learner    = learner,
                    fit_method = pipe.fit(train_df, y_train),
                    get_metric = pipe.score(test_df, y_test),
                    n_trials   = 3
                   )

In [ ]:
#export
print("Number of finished trials: {}".format(len(study.trials)))

In [ ]:
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))

In [ ]:
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
#export
study.trials_dataframe()